<a href="https://colab.research.google.com/github/BraulioRivTri/Pruebas/blob/main/Proyecto1AI(colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import kstest
import statsmodels.regression.linear_model as rg
import statsmodels.stats.diagnostic as dg
from statsmodels.stats.stattools import durbin_watson
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_white
from math import log


In [3]:
insurance=pd.read_csv(r'/content/insurance.csv',delimiter=',')
insurance

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [4]:
#Información de mi base de datos
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [5]:
#Revisión de espacios vacíos
insurance.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [6]:
insurance=pd.get_dummies(insurance,columns=['sex'])
insurance

,age,bmi,children,smoker,region,charges,sex_female,sex_male
0,19,27.900,0,yes,southwest,16884.92400,1,0
1,18,33.770,1,no,southeast,1725.55230,0,1
2,28,33.000,3,no,southeast,4449.46200,0,1
3,33,22.705,0,no,northwest,21984.47061,0,1
4,32,28.880,0,no,northwest,3866.85520,0,1
...,...,...,...,...,...,...,...,...
1333,50,30.970,3,no,northwest,10600.54830,0,1
1334,18,31.920,0,no,northeast,2205.98080,1,0
1335,18,36.850,0,no,southeast,1629.83350,1,0
1336,21,25.800,0,no,southwest,2007.94500,1,0


In [7]:
insurance=pd.get_dummies(insurance,columns=['smoker'])
insurance


,age,bmi,children,region,charges,sex_female,sex_male,smoker_no,smoker_yes
0,19,27.900,0,southwest,16884.92400,1,0,0,1
1,18,33.770,1,southeast,1725.55230,0,1,1,0
2,28,33.000,3,southeast,4449.46200,0,1,1,0
3,33,22.705,0,northwest,21984.47061,0,1,1,0
4,32,28.880,0,northwest,3866.85520,0,1,1,0
...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,northwest,10600.54830,0,1,1,0
1334,18,31.920,0,northeast,2205.98080,1,0,1,0
1335,18,36.850,0,southeast,1629.83350,1,0,1,0
1336,21,25.800,0,southwest,2007.94500,1,0,1,0


In [8]:
insurance=pd.get_dummies(insurance,columns=['region'])
insurance

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,0,1,1,0,0,1,0,0
1334,18,31.920,0,2205.98080,1,0,1,0,1,0,0,0
1335,18,36.850,0,1629.83350,1,0,1,0,0,0,1,0
1336,21,25.800,0,2007.94500,1,0,1,0,0,0,0,1


In [9]:
data=insurance
data

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,0,1,1,0,0,1,0,0
1334,18,31.920,0,2205.98080,1,0,1,0,1,0,0,0
1335,18,36.850,0,1629.83350,1,0,1,0,0,0,1,0
1336,21,25.800,0,2007.94500,1,0,1,0,0,0,0,1


In [15]:
data['log_age']=np.log(data['age'])
data['log_bmi']=np.log(data['bmi'])
data['log_charges']=np.log(data['charges'])
data

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest,log_age,log_bmi,log_charges
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1,2.944439,3.328627,9.734176
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0,2.890372,3.519573,7.453302
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0,3.332205,3.496508,8.400538
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0,3.496508,3.122585,9.998092
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0,3.465736,3.363149,8.260197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,0,1,1,0,0,1,0,0,3.912023,3.433019,9.268661
1334,18,31.920,0,2205.98080,1,0,1,0,1,0,0,0,2.890372,3.463233,7.698927
1335,18,36.850,0,1629.83350,1,0,1,0,0,0,1,0,2.890372,3.606856,7.396233
1336,21,25.800,0,2007.94500,1,0,1,0,0,0,0,1,3.044522,3.250374,7.604867


In [25]:
log_predictors=['age','bmi','children','sex_female','sex_male','smoker_no','smoker_yes','region_northeast','region_northwest','region_southeast','region_southwest']
log_outcome=['log_charges']
log_model=LinearRegression()
log_model.fit(data[log_predictors],data[log_outcome])
print('Intercept',log_model.intercept_)
log_coeficientes=log_model.coef_.tolist()[0]
for name, coef in zip(log_predictors,log_coeficientes):
    print(f'{name}:{coef}')

Intercept [7.68252713]
age:0.034581640531723676
bmi:0.013374822725804442
children:0.10185684855526678
sex_female:0.03770822212164485
sex_male:-0.03770822212164493
smoker_no:-0.7771613929487877
smoker_yes:0.777161392948788
region_northeast:0.08748413289555435
region_northwest:0.023696569257015243
region_southeast:-0.06971261320991577
region_southwest:-0.0414680889426537


In [26]:
model_log=sm.OLS(data[log_outcome],data[log_predictors].assign(const=1))
results_log=model_log.fit()
results_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            log_charges   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.767
Method:                 Least Squares   F-statistic:                     549.8
Date:                Wed, 25 May 2022   Prob (F-statistic):               0.00
Time:                        02:23:46   Log-Likelihood:                -808.52
No. Observations:                1338   AIC:                             1635.
Df Residuals:                    1329   BIC:                             1682.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
age                  0.0346      0.001     39.655      0.000       0.033       0.036
bmi                  0.0134      0.002      6.381      0.000       0.009       0.017
children             0.1019      0.010     10.085      0.000       0.082       0.122
sex_female           1.7449      0.020     88.435      0.000       1.706       1.784
sex_male             1.6695      0.020     82.844      0.000       1.630       1.709
smoker_no            0.9301      0.021     44.948      0.000       0.889       0.971
smoker_yes           2.4844      0.023    108.119      0.000       2.439       2.529
region_northeast     0.9411      0.022     42.754      0.000       0.898       0.984
region_northwest     0.8773      0.022     39.725      0.000       0.834       0.921
region_southeast     0.7839      0.024     32.338      0.000       0.736       0.831
region_southwest     0.8121      0.023     35.639      0.000       0.767       0.857
const                3.4145      0.032    108.219      0.000       3.353       3.476
==============================================================================
Omnibus:                      463.882   Durbin-Watson:                   2.046
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1673.760
Skew:                           1.679   Prob(JB):                         0.00
Kurtosis:                       7.330   Cond. No.                     7.14e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 6.91e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""